In [111]:
import json
import itertools
from bokeh.plotting import figure, show, vplot, gridplot
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, BoxAnnotation, GridPlot
from pykalman import KalmanFilter
import numpy as np

In [112]:
class Data(object):  # each data object contains a time, distance, and rssi value
    
    def __init__(self):
        self.time = 0
        self.distance = 0

In [113]:
laser_times = []
laser_dist = []
rssi_times = []
rssi_dist = []
rssi_rssi = []
beaconNum = ""
initial_time = 0

In [114]:
def plot_data():
    global laser_times
    global laser_dist
    global rssi_times
    global rssi_dist
    global rssi_rssi
    global beaconNum
    temp_laser = []
    temp_rssi = []
    
    for i in range(len(laser_times)):
        for j in range(len(rssi_times)):
            if int(laser_times[i]) == int(rssi_times[j]):  # if occur at the same second
                temp_laser.append(laser_dist[i])  # append two distances
                temp_rssi.append(rssi_rssi[j])
    
    output_notebook()
    p = figure(
        tools="pan, box_zoom, wheel_zoom, reset, resize",
        title=beaconNum, x_axis_label='Laser Distance (m)', y_axis_label='RSSI (dBm)',
        x_range=(0, 10), y_range=(min(temp_rssi), max(temp_rssi)),
        plot_width=900, plot_height=400
    )
    
    # plot measured average distances
    p.scatter(
        temp_laser, temp_rssi,
        marker="circle", fill_color="firebrick", line_color = "firebrick"
    )
    
    '''
    p.line(
        temp_laser, temp_rssi,
        line_color = "navy", legend="Laser Distance"
    )

    
    p.extra_y_ranges = {"RSSI": Range1d(start=min(rssi_dist), end=max(rssi_dist))}  # add y axis for rssi
    p.line(
        rssi_times, rssi_dist,
        line_color = "firebrick", y_range_name="RSSI", legend="Beacon Calc. Distance"
    )
    p.add_layout(LinearAxis(y_range_name="RSSI", axis_label="Beacon Calc. Distance (m)"), 'left')
    '''
    
    show(p)

In [115]:
def get_data(i):  # open text file and extract data
    global initial_time
    global rssi_times
    global rssi_dist
    global rssi_rssi
    global beaconNum
    
    rssi_times[:] = []
    rssi_dist[:] = []
    rssi_rssi[:] = []
    temp = []
    first = True
    prev = 0
    
    if i == 0:
        textName = "0_0.txt"
        beaconNum = "Beacon 0:0 Avg Distance"
    elif i == 1:
        textName = "2_16.txt"
        beaconNum = "Beacon 2:16 Avg Distance"
    else:
        textName = "3_13.txt"
        beaconNum = "Beacon 3:13 Avg Distance"
        
    f = open(textName)

    for line in f:
        words = line.split()
        if first is True:
            initial_time = int(words[0])
            first = False
        parsed_json = json.loads(words[1])  # parse json
        rssi = parsed_json['rssi']
        distance = 10**((rssi+60)/-20)  # log-distance pathloss model
        #info = Data()
        time = (int(words[0]) - initial_time) / 1000
        if time == prev or prev == 0:  # base case or if on same second
            temp.append(rssi)
            prev = time
        else:  # different times
            rssi_times.append(prev)
            rssi_rssi.append(np.mean(temp))  # append avg rssi in that second
            temp[:] = []
            temp.append(rssi)
            prev = time
        
    f.close()  # close file

In [116]:
def laser():
    global initial_time
    global laser_times
    global laser_dist
    temp = []
    first = True
    initial_diff = 0
    prev = 0
    
    f = open("lidar.txt")
    for line in f:
        words = line.split()
        if first is True:
            initial_time = int(words[0])
            first = False
            initial_diff = int(words[1]) - 100  # base value is 100
        #info = Data()
        time = (int(words[0]) - initial_time) / 1000
        distance = (int(words[1]) - initial_diff) / 100
        if time == prev or prev == 0:  # base case or if on same second
            temp.append(distance)
            prev = time
        else:  # different times
            laser_times.append(prev)
            laser_dist.append(np.mean(temp))  # append avg distance in that second
            temp[:] = []
            temp.append(distance)
            prev = time

In [117]:
def main():
    laser() # collect laser data
    for i in range(3):
        get_data(i) # collect other data
        plot_data()
    # plot both

In [118]:
if __name__ == "__main__":
    main()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...